## 一、从高斯消元法到 LU 分解

在第一节中，学习了矩阵乘法的行视点：初等行变换可以用 $B$ 左乘变换矩阵 $A$ 来表示。
在第二节中，讲到线性方程组的求解方法：行简化算法，其实就是对矩阵做一系列的初等行变换。

很自然的，行简化算法，能不能用左乘变换矩阵来表示？

把 $矩阵A$ 变换成 $上三角矩阵U$ 的方法，又称为高斯消元法。它实际上是对矩阵A做了一系列初等行变换，若用 $变换矩阵E$ 表示这个变换，有

$$EA=U$$

我们假设有矩阵 $F$ 使 $FE=I$ 成立，方程左乘 $F$，则有

$$A=FU$$

可以得出的结论是，矩阵 $F$ 是一个下三角矩阵（Lower triangular matirx），一般用 $L$ 表示，于是知道 $矩阵A$可以分解为 $LU$ 的形式。这就叫 LU分解。

**用途**：LU分解常用于求解线性方程组，尤其是在我们需要对各种各样的 $b$ 求解方程组 $Ax=b$ 时，LU分解能节省大量的计算步骤。

### NOTE

1. 注意矩阵A可以为非方阵，若 A 是 $m × n$ 的，则 $L$ 应该是 $m × m$的方阵（变换矩阵E必定为方阵，它的逆L当然也是方阵了），而 $U$ 的形状和 $A$ 相同，为 $ m × n$.
1. 从第一条可知，当 $m < n$ 时，方阵 $L$ 会比较大，这不利于计算。但是实际上我们使用的算法得到的 $L$ 是长方形矩阵，而且也完全不需要求 $E$ 的逆。
1. 通过上述方法得到的矩阵 $L$ 的对角线元素都为 1.

### 举例

从矩阵乘法的行视点来看，对矩阵
$\begin{pmatrix}
     1 & 3 & 1 \\
    1 & 1 & -1 \\
    3 & 11 & 6
\end{pmatrix}$ 做行变换得到 $下三角矩阵U$ 可表示为
$$
\begin{pmatrix}
     1 & 0 & 0 \\
     0 & 1 & 0 \\
     0 & 1 & 1
\end{pmatrix}
\begin{pmatrix}
     1 & 0 & 0 \\
    -1 & 1 & 0 \\
    -3 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
     1 & 3 & 1 \\
    1 & 1 & -1 \\
    3 & 11 & 6
\end{pmatrix} = 
\begin{pmatrix}
     1 & 3 & 1 \\
     0 & -2 & 2 \\
     0 & 0 & 1
\end{pmatrix}
$$
也就是
$$
E_2 E_1 A = U
$$

这样就得到了 $U$。

可以看出 $E$ 也是下三角矩阵： 
$$
E = E_2 E_1 = 
\begin{pmatrix}
     1 & 0 & 0 \\
     -1 & 1 & 0 \\
     -4 & 1 & 1
\end{pmatrix}
$$

接下来要把 $EA=U$ 变换成 $A=LU$ 的形式，也就是求取使 $LE=I$ 成立的 $L$。
显然 $L$ 可以看成是从 $E$ 到 $I$ 的行变换矩阵，应该为
$$
\begin{pmatrix}
     1 & 0 & 0 \\
     1 & 1 & 0 \\
     3 & -1 & 1
\end{pmatrix}
$$

恰好是个下三角矩阵～

所以 $A=LU$ 就是

$$
\begin{pmatrix}
     1 & 3 & 1 \\
    1 & 1 & -1 \\
    3 & 11 & 6
\end{pmatrix} = 
\begin{pmatrix}
     1 & 0 & 0 \\
     1 & 1 & 0 \\
     3 & -1 & 1
\end{pmatrix}
\begin{pmatrix}
     1 & 3 & 1 \\
     0 & -2 & 2 \\
     0 & 0 & 1
\end{pmatrix}
$$

下面用 julia 标准库的函数来验证一下

In [38]:
import LinearAlgebra

A = [1 3  1
     1 1 -1
     3 11 6]

L, U = LinearAlgebra.lu(A, Val(false))  # LU分解

LinearAlgebra.LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
 1.0   0.0  0.0
 1.0   1.0  0.0
 3.0  -1.0  1.0
U factor:
3×3 Array{Float64,2}:
 1.0   3.0   1.0
 0.0  -2.0  -2.0
 0.0   0.0   1.0

得到 $L$ 和 $U$ 之后，矩阵方程 $Ax=b$ 对任意的 $b$，都有 $LUx=b$，令 $Ux=y$，有
$$
\begin{cases}
    Ux=y \\
    Ly=b
\end{cases}
$$
于是解 $Ax=b$ 就变成了解上述两个矩阵方程。乍一看好像要解的方程还变多了，可仔细想想，$L$ 和 $U$ 都已经是三角矩阵了，直接用回代法就能得到 solution!

对任意的 $b$，回代即可得到方程的解，$b$ 越多，节省的计算量就越大！

## 二、矩阵的逆（inverse）

>这里**只讨论方阵的逆**，非方阵只可能存在单侧逆。这里不讨论单侧逆，只是简单地认为非方阵不可逆。

定义：有 $n × n$ 的 $矩阵A$，若存在 $n × n$ 的 $矩阵C$ 使
$$
AC = CA = I_n
$$

成立，则称 $C$ 为 $A$ 的逆，记做 $A^{-1}$。

>P.S. 这里只讨论方阵的逆，非方阵的逆很少用到，而且更复杂，基本没见哪本教材谈过。

不可逆矩阵也称为 **奇异矩阵（singular metrix）**，可逆矩阵称作**非奇异矩阵（non-singular matrix）**

### 2.1 使用矩阵的逆求 $Ax=b$ 的解

如果 $矩阵A$ 可逆，有
$
\boxed{Ax=b} 
\to \boxed{A^{-1}Ax=A^{-1}b}
\to \boxed{x = A^{-1}b}
$

可以看到，$A^{-1}b$ 就是方程的解。

### 2.2 使用矩阵的逆求 LU分解

前面我们已经得出了 $$EA=U \\ E=E_2 E_1$$

于是可以得出 $$A= E^{-1} U = E_1^{-1}E_2^{-1}U$$
于是得到 $$L=E_1^{-1}E_2^{-1}$$

$E_1$ 和 $E_2$ 都是初等矩阵，都可逆，因此只要有 $EA=U$，那 $L$ 一定存在。

>P.S. 这只是理论分析，数值计算中有更好的方法来求 矩阵L。

**NOTE**: **永远不要去求矩阵的逆**，矩阵的逆在理论分析上很有价值，但是在数值计算中，效率低下，基本没人用。解矩阵方程一般都用 LU分解。

### 2.2 Gauss–Jordan 求逆法

这是一种手算方法，权作了解。

该方法首先将 $A$ 与 相同大小的单位阵 $I$ 拼接成增广矩阵
$
\left(
\begin{array}{c|c}
A & I
\end{array}
\right)
$，然后对该增广矩阵进行初等行变换，把左边的 $A$ 变换成 $I$，这个时候，右边的矩阵就是 $A^{-1}$

该方法的原理也很简单，把 $A$ 变换成 $I$ 可以写成如下形式
$$
...E_2E_1A=I
$$

由矩阵的逆的定义 $A^{-1}A=I$ 可知 $A^{-1}=...E_2E_1$

在对增广矩阵做初等行变换时，不仅 $A$ 做了该变换，右边的 $I$ 也做了同样的变换，于是右边应该变成了 $...E_2E_1I=A^{-1}I=A^{-1}$，因此右边就是矩阵的逆。

### NOTE

1. **可逆的条件**: 从上面可以看出，矩阵A 的逆矩阵 $A^{-1}$ 要存在，就是说 $A^{-1}=...E_2E_1$ 要存在，也就是说**要能通过一系列的初等行变换，把 $A$ 变换成 $I$，即矩阵 $A$ 行等价于单位矩阵 $I$**。**首先把 $A$ 化成阶梯形式，如果它有全为零的行**，这个矩阵就不可能通过对 $I$ 做行变换得到（行变换的数乘要求系数不为0），所以它**一定不可逆**。
1. 这里只讨论方阵的逆，方阵化为阶梯形式后，若不存在全为0的行，那它对角线上的元素一定都不为0！也就是说**$n × n$ 的矩阵 $A$ 可逆，等价于它有n个主元列，或者说 $A$ 不存在自由变量。**
1. 也可以这样拼接：$
\begin{pmatrix}
A \\
I
\end{pmatrix}
$，然后做初等列变换，也能得到同样的 $A^{-1}$